# Segment, localise and track


Fixing a bug in the code that meant the intracellular mean intensity values weren't accurate due to the downscaling

In [2]:
from macrohet import dataio, tile
from macrohet.notify import send_sms
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
from octopuslite import utils
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
# defining cellpose model to use
model = models.Cellpose(
                        gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.1 GB


### Define functions to tidy up main block of code

In [3]:
def segment(frame, model = model, channels = [0,0], diameter = 325, min_size = 2500):
    
    masks, flows, styles, diams = model.eval(frame, 
                                             channels = channels, 
                                             diameter = diameter, 
                                             min_size = min_size, 
                                             )
    return masks

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

scale_factor = 1/5.04

def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects

# define config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/btrack/models/particle_config_pnassign.json'

def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [4]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'

In [5]:
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [6]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Segment, localise and track

In [7]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')

In [8]:
input_transforms = utils.remove_background

In [8]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 input_transforms = [input_transforms]
                                 ).astype(np.uint16)

    # reload seg
    tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
    with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
        masks = hdf.segmentation

    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 20)

    # save out 
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/relocalised_bg_rm_{row, column}.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
#             writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
    print(f"Checking time indices: {row, column, tracks[0]['t']}")
    send_sms(f"Position {row, column} bg rm, retracked and saved out, {tracks[0]['t']}")

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/05/02 06:19:12 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 4)_objs_masks_tracks.h5...
[INFO][2023/05/02 06:19:26 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/02 06:19:26 PM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 4)_objs_masks_tracks.h5
[INFO][2023/05/02 06:19:26 PM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████| 75/75 [2:00:18<00:00, 96.25s/it]
[INFO][2023/05/02 08:19:44 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/02 08:19:45 PM] ...Found 43409 objects in 75 frames.
[INFO][2023/05/02 08:19:45 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/02 08:19:45 PM] Starting BayesianTracker session
[INFO][2023/05/02 08:19:45 PM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/02 08:19:45 PM] Objects are of type: <class 'list'>
[INFO][2023/05/02 08:19:45

GLPK Integer Optimizer 5.0
12636 rows, 10776 columns, 15234 non-zeros
10776 integer variables, all of which are binary
Preprocessing...
6318 rows, 10776 columns, 15234 non-zeros
10776 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6318
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
6318 rows, 10776 columns, 15234 non-zeros
*     0: obj =   3.245579138e+04 inf =   0.000e+00 (2584)
Perturbing LP to avoid stalling [1250]...
Removing LP perturbation [2587]...
*  2587: obj =   1.656273218e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2587: mip =     not found yet >=              -inf        (1; 0)
+  2587: >>>>>   1.656273218e+04 >=   1.656273218e+04   0.0% (1; 0)
+  2587: mip =   1.656273218e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2023/05/02 08:20:04 PM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_bg_rm_(3, 4).h5...
[INFO][2023/05/02 08:20:05 PM] Writing objects/obj_type_1
[INFO][2023/05/02 08:20:05 PM] Writing labels/obj_type_1
[INFO][2023/05/02 08:20:05 PM] Loading objects/obj_type_1 (26904, 5) (26904 filtered: None)
[INFO][2023/05/02 08:20:05 PM] Writing properties/obj_type_1/area (26904,)
[INFO][2023/05/02 08:20:05 PM] Writing properties/obj_type_1/major_axis_length (26904,)
[INFO][2023/05/02 08:20:05 PM] Writing properties/obj_type_1/minor_axis_length (26904,)
[INFO][2023/05/02 08:20:05 PM] Writing properties/obj_type_1/orientation (26904,)
[INFO][2023/05/02 08:20:05 PM] Writing properties/obj_type_1/mean_intensity (26904, 2)
[INFO][2023/05/02 08:20:05 PM] Writing tracks/obj_type_1
[INFO][2023/05/02 08:20:05 PM] Writing dummies/obj_type_1
[INFO][2023/05/02 08:20:05 PM] Writing LBEP/obj_type_1
[INFO][2023/05/02 08:20:05 PM] Writing fates/obj_type_1
[INFO][2023/05/02 08:20:49

Checking time indices: (3, 4, [0, 1, 2])


...

[INFO][2023/05/03 09:58:13 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(4, 7)_objs_masks_tracks.h5...
[INFO][2023/05/03 09:58:22 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/03 09:58:22 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(4, 7)_objs_masks_tracks.h5
[INFO][2023/05/03 09:58:22 AM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████| 75/75 [1:35:23<00:00, 76.31s/it]
[INFO][2023/05/03 11:33:45 AM] Objects are of type: <class 'dict'>
[INFO][2023/05/03 11:33:46 AM] ...Found 47373 objects in 75 frames.
[INFO][2023/05/03 11:33:46 AM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/03 11:33:46 AM] Starting BayesianTracker session
[INFO][2023/05/03 11:33:46 AM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/03 11:33:46 AM] Objects are of type: <class 'list'>
[INFO][2023/05/03 11:33:48

GLPK Integer Optimizer 5.0
13000 rows, 11036 columns, 15572 non-zeros
11036 integer variables, all of which are binary
Preprocessing...
6500 rows, 11036 columns, 15572 non-zeros
11036 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6500
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
6500 rows, 11036 columns, 15572 non-zeros
*     0: obj =   3.336032086e+04 inf =   0.000e+00 (2680)
Perturbing LP to avoid stalling [1240]...
Removing LP perturbation [2667]...
*  2667: obj =   1.721354786e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2667: mip =     not found yet >=              -inf        (1; 0)
+  2667: >>>>>   1.721354786e+04 >=   1.721354786e+04   0.0% (1; 0)
+  2667: mip =   1.721354786e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2023/05/03 11:34:26 AM] Ending BayesianTracker session
[INFO][2023/05/03 11:34:27 AM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_bg_rm_(4, 7).h5...
[INFO][2023/05/03 11:34:28 AM] Writing objects/obj_type_1
[INFO][2023/05/03 11:34:28 AM] Writing labels/obj_type_1
[INFO][2023/05/03 11:34:28 AM] Loading objects/obj_type_1 (27613, 5) (27613 filtered: None)
[INFO][2023/05/03 11:34:28 AM] Writing properties/obj_type_1/area (27613,)
[INFO][2023/05/03 11:34:28 AM] Writing properties/obj_type_1/major_axis_length (27613,)
[INFO][2023/05/03 11:34:28 AM] Writing properties/obj_type_1/minor_axis_length (27613,)
[INFO][2023/05/03 11:34:28 AM] Writing properties/obj_type_1/orientation (27613,)
[INFO][2023/05/03 11:34:28 AM] Writing properties/obj_type_1/mean_intensity (27613, 2)
[INFO][2023/05/03 11:34:28 AM] Writing tracks/obj_type_1
[INFO][2023/05/03 11:34:28 AM] Writing dummies/obj_type_1
[INFO][2023/05/03 11:34:28 AM] Writing LBEP/obj_type_1
[INFO][2023/05/03 11

Checking time indices: (4, 7, [0])


[INFO][2023/05/03 11:36:01 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(4, 8)_objs_masks_tracks.h5...
[INFO][2023/05/03 11:36:23 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/03 11:36:23 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(4, 8)_objs_masks_tracks.h5
[INFO][2023/05/03 11:36:23 AM] Localizing objects from segmentation...
  1%|▊                                                            | 1/75 [03:04<3:47:40, 184.60s/it]
KeyboardInterrupt

